# Dynamic & automated NER Data Gathering and Processing
## Steps
 - Gathering Job Description data from 5 different sources in kaggle
 - Find all words that are a kind of job skill in all those texts
 - Label all finding words in a common NER format

 ## The Idea of Dynamic Labelling Comes from [Here](https://mehdihosseinimoghadam.github.io//posts/2022/04/Punctuation-and-Capitalization/) & [Here](https://github.com/kaliani/ner_detect)

In [1]:
pip install -q kaggle

# You need to upload your kaggle json credentials here

# More [here](https://www.kaggle.com/discussions/general/74235)

In [2]:
!mkdir ~/.kaggle/

In [3]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d cedricaubin/linkedin-data-analyst-jobs-listings
!kaggle datasets download -d JobsPikrHQ/usa-based-job-data-set-from-300-companies
!kaggle datasets download -d shivamb/real-or-fake-fake-jobposting-prediction
!kaggle datasets download -d thedevastator/jobs-dataset-from-glassdoor
!kaggle datasets download -d andrewmvd/data-analyst-jobs
!kaggle datasets download -d arshkon/linkedin-job-postings

 58% 5.00M/8.63M [00:00<00:00, 49.4MB/s]
100% 8.63M/8.63M [00:00<00:00, 76.1MB/s]
 40% 5.00M/12.4M [00:00<00:00, 41.3MB/s]
100% 12.4M/12.4M [00:00<00:00, 79.2MB/s]
 56% 9.00M/16.1M [00:00<00:00, 76.8MB/s]
100% 16.1M/16.1M [00:00<00:00, 113MB/s] 
  0% 0.00/3.23M [00:00<?, ?B/s]
100% 3.23M/3.23M [00:00<00:00, 131MB/s]
  0% 0.00/2.25M [00:00<?, ?B/s]
100% 2.25M/2.25M [00:00<00:00, 129MB/s]
 51% 11.0M/21.5M [00:00<00:00, 111MB/s]
100% 21.5M/21.5M [00:00<00:00, 164MB/s]


In [5]:
!unzip /content/data-analyst-jobs.zip
!unzip /content/jobs-dataset-from-glassdoor.zip
!unzip /content/linkedin-data-analyst-jobs-listings.zip
!unzip /content/linkedin-job-postings.zip
!unzip /content/real-or-fake-fake-jobposting-prediction.zip
!unzip /content/usa-based-job-data-set-from-300-companies.zip

Archive:  /content/data-analyst-jobs.zip
  inflating: DataAnalyst.csv         
Archive:  /content/jobs-dataset-from-glassdoor.zip
  inflating: eda_data.csv            
  inflating: glassdoor_jobs.csv      
  inflating: salary_data_cleaned.csv  
Archive:  /content/linkedin-data-analyst-jobs-listings.zip
  inflating: linkedin-jobs-africa.csv  
  inflating: linkedin-jobs-canada.csv  
  inflating: linkedin-jobs-usa.csv   
Archive:  /content/linkedin-job-postings.zip
  inflating: company_details/companies.csv  
  inflating: company_details/company_industries.csv  
  inflating: company_details/company_specialities.csv  
  inflating: company_details/employee_counts.csv  
  inflating: job_details/benefits.csv  
  inflating: job_details/job_industries.csv  
  inflating: job_details/job_skills.csv  
  inflating: job_postings.csv        
Archive:  /content/real-or-fake-fake-jobposting-prediction.zip
  inflating: fake_job_postings.csv   
Archive:  /content/usa-based-job-data-set-from-300-companies

## This drive file is a list of skills extracted fro different sources which will be used to find skill in job descriptions

In [6]:
!gdown --i 1x_OKBDINO3H3_JlGFruhY0dx44EZhVu3

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1x_OKBDINO3H3_JlGFruhY0dx44EZhVu3
To: /content/all_gathered_skills.csv
100% 33.2k/33.2k [00:00<00:00, 81.0MB/s]


In [7]:
import pandas as pd
df1 = pd.read_csv('/content/US-based jobs.csv')
df2 = pd.read_csv('/content/DataAnalyst.csv')
df3 = pd.read_csv('/content/fake_job_postings.csv')
df4 = pd.read_csv('/content/glassdoor_jobs.csv')
df5 = pd.read_csv('/content/job_postings.csv')


In [8]:
df1['JD'] = df1['JobText']
df2['JD'] = df2['Job Description']
df3['JD'] = df3['description']
df4['JD'] = df4['Job Description']
df5['JD'] = df5['description']


all_job_text = pd.concat([df1['JD'], df2['JD'], df3['JD'], df4['JD'], df5['JD']], axis=0).to_list()
all_job_text = pd.DataFrame(all_job_text, columns=['JD'])
all_job_text

,JD
0,21 Tech is looking for a Legal Editor to work ...
1,21 Tech is looking for a Business Data Analyst...
2,Talent Table a 21 Tech Subsidiary is looking f...
3,21Tech is looking for a Senior Infor EAM Funct...
4,21 Tech is looking for a Legal Editor to work ...
...,...
61346,"Location:\n\nWest Columbia, SC, US, 29172\n\n2..."
61347,Job Title: Unit Secretary\nDepartment: Nursing...
61348,"Job Title: Radiology Aide, Perdiem\nDepartment..."
61349,Grade 105\nJob Type: Officer of Administration...


## Read all skills (words that are skill like: project management, NLP, A.I. ...) and doing some preprocessing on them and sorting



## Here each of the job descriptions from our database will be iterated
### First it will be splited by . so we have shorter length sentences then some puntuation removal will be added as a preprocessing (I won't use stemming or lemmitization because a user can write anything and I don't want to chande the structure of text too much)


then the sentence will be given to `transform_to_label` function to check if any of the skill words that we loaded previously (entities list) are present in the sentence, if so it will label the skill as `In-Sk` otherwise label as `O`

so sentence 'We need someone with statistical background and fluent in python' will be:

'O,O,O,O,In-Sk,O,O,O,O,In-Sk'

In [9]:
import re
import csv

def text_processing(j):
  '''
  To preProcess the input text, remove puntuation and special words

  '''
  j.replace("c++", "c+-")
  j = re.sub(r'A/B testing','A/Btesting',j)
  j = re.sub(r'\r','',j)
  j = re.sub(r'\n','',j)
  j = re.sub(r'\t','',j)
  j = re.sub(r'\xa0','',j)           # C++ makes problem in regex so use replace instead
  j = re.sub(r'/','SPECIAL1',j)      # To keep the / in text corpus for words like A/B testing
  j = re.sub(r'[^\w\s]',' ',j)
  j = re.sub('e g',' ',j)
  j = re.sub(r'SPECIAL1','/',j)
  return j


def parse_skills(text):
    """Extract user skills from text and return as a list."""
    with open('/content/all_gathered_skills.csv', 'r') as f:
        reader = csv.reader(f)
        skill_list = [row[1].lower() for row in reader]

    skill_regex = r"\b(" + "|".join(skill_list) + r")\b"
    return re.findall(skill_regex, text)



def all_skills(parsed_skills):
  '''
  Break all skills into words some of skills like 'project management' has two or several words and can
  be ttrricky when it comes to matching in label_text function
  '''
  l = []
  for i in parsed_skills:
    l.extend(i.split())
  return l



def labeled_text(text,skills):
    if  not skills:
        return " ".join(["O"] * len(text.split()))
    tags = []
    for word in text.split():
        if word.lower() in skills:
            tags += ["In-SK"]
        else:
            tags += ["O"]
    return ",".join(tags)

In [10]:
from tqdm import tqdm
import re

def text_processing(j):
  j = re.sub(r'A/B testing','A/Btesting',j)
  j = re.sub(r'\r','',j)
  j = re.sub(r'\n','',j)
  j = re.sub(r'\t','',j)
  j = re.sub(r'\xa0','',j)
  j = re.sub(r'/','SPECIAL1',j)      # To keep the / in text corpus for words like A/B testing
  j = re.sub(r'[^\w\s]',' ',j)
  j = re.sub('e g',' ',j)
  j = re.sub(r'SPECIAL1','/',j)
  return j


def transform_to_label(text):
  Os = []
  for q in text.split():
    if q == 'In-SK':
      Os.append('In-SK')
    else:
      Os.append('O')
  return ','.join(Os)


def transform_to_label1(text):                                # some parts taken from https://github.com/kaliani/ner_detect
  sk = parse_skills(text_processing(text.lower()))
  return(labeled_text(text_processing(text.lower()), all_skills(sk)))



In [11]:
list_of_all_labeled_data = []

for k in tqdm(range(0,500)):                             # Control how many data samples you want to label
  try:
    job = list(set(all_job_text['JD'][k].split('. ')))
    for j in job:
      l = []
      j = text_processing(j)
      jj = j
      if 'In-SK' in transform_to_label1(j):
        l.append(jj)
        l.append(transform_to_label1(j))
        list_of_all_labeled_data.append(l)
  except:
    pass

100%|██████████| 500/500 [00:25<00:00, 19.92it/s]


In [ ]:
#old time intensive method

# list_of_all_labeled_data = []

# for k in tqdm(range(0,10)):                             # Control how many data samples you want to label
#   # try:
#     job = list(set(all_job_text['JD'][k].split('. ')))
#     for j in job:
#       l = []
#       j = text_processing(j)
#       jj = j
#       for i in entities:
#         j = re.sub(i , ' In-SK '*len(i.split()), j)    #   * len because some of them has several words
#       if 'In-SK' in transform_to_label(j):
#         l.append(jj)
#         l.append(transform_to_label(j))
#         list_of_all_labeled_data.append(l)
#   # except:
#   #   pass

In [12]:
print(len(list_of_all_labeled_data[10][0].split()))
print(len(list_of_all_labeled_data[10][1].split(',')))

41
41


In [13]:
print(list_of_all_labeled_data[4])
print(list_of_all_labeled_data[7])

['You must be authorized to work in the United States without sponsorship  No H1B Day to day operations include analysis of weather data  past and forecast   in relation to various sectors  including agriculture  corn  soybeans  wheat  and energy  power and gas ', 'O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,O,O']
['Responsibilities include project planning  scope management  project change control  schedule and budget management  requirements definition  COTS software selections  software implementation  integration  conversion  training  deployment and continuing support  Essential Duties and Responsibilities Conduct interviews with stakeholders to assess use of current system Lead business process sessions to document As Is processes Develop Functional and technical requirements for To Be business processes Maintain excellent client relations  meets customer expectations for qualityDesired Skills and Experience 5 plus years of technology rel

In [14]:
data = pd.DataFrame(list_of_all_labeled_data, columns = ["sentence", "word_labels"])
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data

,sentence,word_labels
0,This includes reviewing large numbers of docum...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In..."
1,Will report directly to Head of Weather Resear...,"O,O,O,O,O,O,O,In-SK,O,O,O,O,O"
2,A wide degree of creativity and latitude is ex...,"O,O,O,O,In-SK,O,O,O,O"
3,The analyst will be expected to interpret resu...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In..."
4,You must be authorized to work in the United S...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,..."
...,...,...
2051,Would Be Great Extensive experience in Java S...,"O,O,O,O,O,O,In-SK,O,In-SK,In-SK,O,O,O,O,In-SK,..."
2052,Human Resources training and experience desi...,"In-SK,In-SK,In-SK,O,O,O,O,O,O,O,In-SK,O,O,O,O,..."
2053,MS/MBA Engineering Manufacturing or Opera...,"O,In-SK,In-SK,O,In-SK,In-SK,O,O,O,O,O,O,O,O,O,..."
2054,group of companies and is an Equal Opportunity...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [54]:
data.to_csv('full_data_125k.csv')

# Full Training data contains ~125000 labeled sentences
